# Install

In [ ]:
!git clone https://github.com/farzane-maghsoudi/TTL-GAN
import os
os.chdir('TTL-GAN/')
!pip install -r requirements.txt
!bash ./download_dataset.sh horse2zebra

#dataset

In [ ]:
!bash ./download_dataset.sh horse2zebra

#Training

In [ ]:
!python main.py --dataset horse2zebra --light True

Restoring from the previous checkpoint

In [ ]:
!python main.py --dataset horse2zebra --resume True --light True

#Testing

In [ ]:
!python main.py --dataset mini --phase test --light True

#Copy checkpoint to GoogleDrive and convert




In [ ]:
from google.colab import drive
drive.mount("/content/gdrive")

Mounted at /content/gdrive


In [ ]:
!mkdir ./results

In [ ]:
b = "/content/gdrive/MyDrive/checkpointTTL-GAN/horse2zebra_params_latest.pt"
c = "/content/TTL-GAN/results/horse2zebra_params_latest.pt"
f1 = "/content/gdrive/MyDrive/checkpointTTL-GAN/horse2zebra_params_0002000.pt"
f2 = "/content/TTL-GAN/results/horse2zebra/model/horse2zebra_params_0002000.pt"
cc = "/content/TTL-GAN/results/horse2zebra_params_0002000.pt"

Colab to Drive

In [ ]:
import shutil
shutil.copy( c , b )
shutil.copy( f2 , f1 )

Drive to Colab

In [ ]:
import shutil
shutil.copy( f1 , cc )

'/content/TTL-GAN/results/horse2zebra_params_0002000.pt'

#Test save

In [ ]:
#import shutil
#b = "/content/gdrive/MyDrive/results/501/UGATIT_light_apple2orange_lsgan_4resblock_6dis_1_1_10_10_1000_1_1_1_sn_smoothing"
#c = "/content/UG/results/UGATIT_light_apple2orange_lsgan_4resblock_6dis_1_1_10_10_1000_1_1_1_sn_smoothing"
!cp "/content/UG/results/UGATIT_light_apple2orange_lsgan_4resblock_6dis_1_1_10_10_1000_1_1_1_sn_smoothing" -r "/content/gdrive/MyDrive/results"

In [ ]:
n_blocks = 8
for i in range(2, n_blocks+1):
  if i%3 == 0:
    x2 = 3
    print (i, x2)
  if i%3 == 1:
    x3 = 4
    print (i, x3)
  if i%3 == 2:
    if i> 4:
      x4 = 10
      print (i, x4)
    else:
      x4 = 5
      print (i, x4)

In [ ]:
import torch
import torch.nn as nn
a = torch.randn(1,256, 64, 64)
#a = torch.reshape(a, (256, 64, 64))
atten  = torch.nn.MultiheadAttention(64, 8)
relu = torch.nn.ReLU(True)
class MultiSelfAttentionBlock(nn.Module):
    def __init__(self, atten, relu, dim =64, featur = 256):
      super(MultiSelfAttentionBlock, self).__init__()
      self.dim = dim
      self.featur = featur
      self.atten  = atten
      self.relu  = relu


    def forward(self, x):
      print("as")
      out = torch.reshape(x, (self.featur, self.dim, self.dim))
      out, _ = self.atten(out, out, out)
      out = self.relu(torch.reshape(out, (1, self.featur, self.dim, self.dim)))
      print("a")

      return out

#out, _ = atten(a, a, a)
#out = torch.reshape(out, (1, 256, 64, 64))
for i in range(3):
  out = MultiSelfAttentionBlock(a, atten, relu)
  #print (out.shape)

In [ ]:
import torch
import torch.nn as nn
a = torch.randn(1,256, 64, 64)
atten  = torch.nn.MultiheadAttention(64, 8)
relu = torch.nn.ReLU(True)
for i in range(3):
  print("as")
  out = torch.reshape(a, (256, 64, 64))
  out, _ = atten(out, out, out)
  out = torch.reshape(out, (1, 256, 64, 64))
  out = relu(out)
  print("a")
  print (out.shape)

In [ ]:
import torch
import torch.nn as nn
x = torch.randn(1,256, 64, 64)
atten  = torch.nn.MultiheadAttention(64, 8)
relu = torch.nn.ReLU(True)
n_blocks = 8
out = torch.reshape(x, (256, 64, 64))
out, _ = atten(out, out, out)
out = relu(torch.reshape(out, (1, 256, 64, 64)))
xa1 = out
xa11 = out
if n_blocks>1:
  for i in range(2, 8+1):
    if i%3 == 2:
      out = torch.reshape(out, (256, 64, 64))
      out, _ = atten(out, out, out)
      xa2 = out = relu(torch.reshape(out, (1, 256, 64, 64)))
      print (i)
    elif i%3 == 0:
      out = torch.reshape(out + xa1, (256, 64, 64))
      out, _ = atten(out, out, out)
      xa3 = out = relu(torch.reshape(out, (1, 256, 64, 64)))
      print (i)
    elif i < 6:
      out = torch.reshape(out + xa1 + xa2, (256, 64, 64))
      out, _ = atten(out, out, out)
      xa1 = out = relu(torch.reshape(out, (1, 256, 64, 64)))
      print (i)
    else:
      out = torch.reshape(out + xa1 + xa2 + xa11, (256, 64, 64))
      out, _ = atten(out, out, out)
      xa11 = xa1
      xa1 = out = relu(torch.reshape(out, (1, 256, 64, 64)))
      print (i)

In [ ]:
import torch
import torch.nn as nn
a = torch.randn(1,128, 56, 56)
b = torch.randn(1,128, 28, 28)
c = torch.randn(1,128, 14, 14)
up1 = nn.Upsample(scale_factor=2, mode='bilinear', align_corners=True)
up2 = nn.Upsample(scale_factor=4, mode='bilinear', align_corners=True)
pad1 = nn.ReflectionPad2d(4)
pad2 = nn.ReflectionPad2d(4)
pad3 = nn.ReflectionPad2d(4)
a = pad1 (a)
b = up1(b)
b = pad2(b)
c = up2(c)
c = pad3(c)
print (a.shape)
print(b.shape)
print(c.shape)

In [ ]:
self.save(self.checkpoint_dir, counter)
				    a = (epoch+1)*1000
				    b = "/content/gdrive/MyDrive/checkpoint/UGATIT_light_apple2orange_lsgan_4resblock_6dis_1_1_10_10_1000_1_1_1_sn_smoothing"
				    c = "/content/UG/checkpoint/UGATIT_light_apple2orange_lsgan_4resblock_6dis_1_1_10_10_1000_1_1_1_sn_smoothing"
				    f1 = "/UGATIT_light.model-"+ str(a)+ ".index"
				    f2 = "/UGATIT_light.model-"+ str(a)+ ".meta"
				    f3 = "/UGATIT_light.model-" + str(a)+ ".data-00000-of-00001"
				    f = "/checkpoint"
				    import shutil
				    shutil.copy( c+f1 , b+f1 )
				    shutil.copy( c+f2 , b+f2 )
				    shutil.copy( c+f3 , b+f3 )
				    shutil.copy( c+f , b+f )

In [43]:
import torch
import torch.nn as nn
a = torch.randn(1,4, 4,3)
s1 = torch.linalg.svdvals(a)
out = torch.reshape(s1[0,:,0], (-1,))
#print(out.shape)
print(s1.shape)
print(s1)
print(out)
#print(out[:,1,1])

#b = torch.cartesian_prod()

torch.Size([1, 4, 3])
tensor([[[3.0127, 1.5599, 0.6251],
         [3.5843, 1.2355, 0.8674],
         [2.9782, 2.0144, 0.9867],
         [2.5381, 2.2221, 1.6962]]])
tensor([3.0127, 3.5843, 2.9782, 2.5381])
